In [2]:

import networkx as nx


In [3]:
# gives shortest dist for all nodes to all other nodes
def floyd_warshall(graph:nx.Graph):
    distances = {}
    nodes = list(graph.nodes)

    # initalise 2 key dict populated with inf distance
    for node1 in nodes:
        distances[node1]={}
        for node2 in nodes:
            distances[node1][node2] = float("inf")

    # set dist to nodes themselves to 0
    for node in nodes:
        distances[node][node]=0

    # set dist for every given edge
    # aka populates dist for adjacency
    # diffrence is for digraph it only populates in direction of one
    for edge in graph.edges.data():
            node1, node2, pdict = edge
            weight = pdict["weight"]
            distances[node1][node2]=weight
            distances[node2][node1]=weight
    
    # up to here distances is an adjacency matric as a dualkey dictionary
    # print(distances)


    # important for intermedate node loop to be on outside (?) might be absolutely nessicary for relaxation
    for intermediaryNode in nodes:
        for nodeI in nodes:
            for nodeJ in nodes:
                # sees if using intermediaryNode between nodeI to nodeJ is faster, if so add
                distances[nodeI][nodeJ] = min(distances[nodeI][nodeJ], distances[nodeI][intermediaryNode] + distances[intermediaryNode][nodeJ])

                # disrespect direction
                distances[nodeI][nodeJ] = min(distances[nodeI][nodeJ],distances[nodeI][nodeJ])


    # if still unresolved -> negative cycle
    for intermediaryNode in nodes:
        for nodeI in nodes:
            for nodeJ in nodes:
                if distances[nodeI][intermediaryNode] + distances[intermediaryNode][nodeJ] < distances[nodeI][nodeJ] :
                    return "negative cycle"

    
    return distances


In [4]:
# single node source  
# good for graphs with no negative edges

def dijkstra(graph: nx.Graph, start) -> dict:

    unvisited = list(graph.nodes)
    # gets list

    node_distances = {node: float('inf') for node in graph.nodes}
    # creates set of distances, default initiate to inf
    
    node_distances[start] = 0

    # while there is still nodes to be visited
    while len(unvisited) > 0:
        
        # set current node to a unvisited node with shortest dist to start
        node = min(unvisited, key=lambda x: node_distances[x])
    
        # 
        for i in graph.neighbors(node):
            # checks all unvisited neighbours of currrent node
            if i in unvisited:
    

                # checks if using current node as a intermediary will result in faster route
                node_distances[i] = min(node_distances[i], node_distances[node] + graph[node][i]['weight'])
                
                # disrespect direction
                node_distances[i] = min(node_distances[i], node_distances[node] + graph[i][node]['weight'])
    
        unvisited.remove(node)
    
    return node_distances




In [5]:
# single source
# good for graphs with negative edges (directed only) 
#  as all undir edges are cycles by themselves, undir cannot handle negative lengh
# will return if detects negative cycle 



def bellman_ford(graph: nx.Graph,start):
    if start not in graph.nodes:
        raise Exception("start node not in grpah")
    distances={}
    for node in graph.nodes:
        distances[node]=float('inf')
    distances[start]=0


    for _ in range(len(graph.nodes)-1):
        for edge in graph.edges.data():
            nodeU,nodeV,pdict=edge
            weight=pdict["weight"]

            if distances[nodeU] + weight < distances[nodeV]:
                distances[nodeV]=distances[nodeU] + weight

            # make it disrespect direction
            if distances[nodeV] + weight < distances[nodeU]:
                distances[nodeU]=distances[nodeV] + weight


    # detect negative cycle
    # basically, the graph will be fully relaxed/finalised by now as we relaexd |V|-1 times
    # so if there is still any futher possible shortest path, that means theres a negative cycle
    for edge in graph.edges.data():
            nodeU,nodeV,pdict=edge
            weight=pdict["weight"]

            if distances[nodeU] + weight < distances[nodeV]:
                return "negative cycle"

            # make it disrespect direction
            if distances[nodeV] + weight < distances[nodeU]:
                return "negative cycle"

    
    return distances


In [6]:
# testing

nodes=['a', 'b', 'c', 'd', 'e', 'f']
edges=[('a', 'b', {'weight': 6}), ('a', 'd', {'weight': 5}), ('a', 'c', {'weight': 1}), ('b', 'c', {'weight': 5}), ('d', 'c', {'weight': 5}), ('b', 'e', {'weight': 3}), ('e', 'f', {'weight': 6}), ('e', 'c', {'weight': 6}), ('c', 'f', {'weight': 4}), ('d', 'f', {'weight': 2})]

# edges=[ ('a', 'b', {'weight': 5}), ('a', 'b', {'weight': 1}), ('b', 'c', {'weight': 5})]


ingraph=nx.Graph()


ingraph.add_nodes_from(nodes)

ingraph.add_edges_from(edges)

# tests=["a","c","d"]
tests=nodes
for testvar in tests:
    print(f"test souuce node {testvar}")
    print(floyd_warshall(ingraph)[testvar])
    print(dijkstra(ingraph,testvar))
    print(bellman_ford(ingraph,testvar))
    print()

test souuce node a
{'a': 0, 'b': 6, 'c': 1, 'd': 5, 'e': 7, 'f': 5}
{'a': 0, 'b': 6, 'c': 1, 'd': 5, 'e': 7, 'f': 5}
{'a': 0, 'b': 6, 'c': 1, 'd': 5, 'e': 7, 'f': 5}

test souuce node b
{'a': 6, 'b': 0, 'c': 5, 'd': 10, 'e': 3, 'f': 9}
{'a': 6, 'b': 0, 'c': 5, 'd': 10, 'e': 3, 'f': 9}
{'a': 6, 'b': 0, 'c': 5, 'd': 10, 'e': 3, 'f': 9}

test souuce node c
{'a': 1, 'b': 5, 'c': 0, 'd': 5, 'e': 6, 'f': 4}
{'a': 1, 'b': 5, 'c': 0, 'd': 5, 'e': 6, 'f': 4}
{'a': 1, 'b': 5, 'c': 0, 'd': 5, 'e': 6, 'f': 4}

test souuce node d
{'a': 5, 'b': 10, 'c': 5, 'd': 0, 'e': 8, 'f': 2}
{'a': 5, 'b': 10, 'c': 5, 'd': 0, 'e': 8, 'f': 2}
{'a': 5, 'b': 10, 'c': 5, 'd': 0, 'e': 8, 'f': 2}

test souuce node e
{'a': 7, 'b': 3, 'c': 6, 'd': 8, 'e': 0, 'f': 6}
{'a': 7, 'b': 3, 'c': 6, 'd': 8, 'e': 0, 'f': 6}
{'a': 7, 'b': 3, 'c': 6, 'd': 8, 'e': 0, 'f': 6}

test souuce node f
{'a': 5, 'b': 9, 'c': 4, 'd': 2, 'e': 6, 'f': 0}
{'a': 5, 'b': 9, 'c': 4, 'd': 2, 'e': 6, 'f': 0}
{'a': 5, 'b': 9, 'c': 4, 'd': 2, 'e': 6, '